# Sign Language Dataset Scikit

The Sign Language Dataset consists of 9680 grayscale images of hand signs for the digits 0-9 and the alphabets a-z. Thus, this is a multiclass classification problem with 36 classes. Your task is to build a machine learning model that can accurately classify images from this dataset.

## Loading the dataset

You **do not** need to upload any data. Both the visible training dataset and the hidden test dataset are already available on the Jupyter hub.

In [1]:
import os
import csv
import cv2
import random
import numpy as np
import matplotlib.pyplot as plt

Matplotlib created a temporary config/cache directory at /var/folders/9l/hc6vjb_16sj0br4xnxqpwcxw0000gn/T/matplotlib-bchtu_po because the default path (/Users/thomas/.matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [5]:
# Setting the path of the training dataset (that was already provided to you)

running_local = True if os.getenv('JUPYTERHUB_USER') is None else False
DATASET_PATH = "."

# Set the location of the dataset
if running_local:
    # If running on your local machine, the sign_lang_train folder's path should be specified here
    local_path = "sign_lang_train"
    if os.path.exists(local_path):
        DATASET_PATH = local_path
else:
    # If running on the Jupyter hub, this data folder is already available
    # You DO NOT need to upload the data!
    DATASET_PATH = "/data/mlproject21/sign_lang_train"

In [6]:
# Utility function

def read_csv(csv_file):
    with open(csv_file, newline='') as f:
        reader = csv.reader(f)
        data = list(reader)
    return data

## Data Loading using PyTorch

For creating and training your model, you can work with any machine learning library of your choice. 

If you choose to work with [PyTorch](https://pytorch.org/), you will need to create your own [Dataset](https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset) class for loading the data. This is provided below. See [here](https://pytorch.org/tutorials/beginner/data_loading_tutorial.html) for a nice example of how to create a custom data loading pipeline in PyTorch. 

In [7]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, utils, io
from torchvision.utils import make_grid

from string import ascii_lowercase


class SignLangDataset(Dataset):
    """Sign language dataset"""

    def __init__(self, csv_file, root_dir, class_index_map=None, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.data = read_csv(os.path.join(root_dir, csv_file))
        self.root_dir = root_dir
        self.class_index_map = class_index_map
        self.transform = transform
        # List of class names in order
        self.class_names = list(map(str, list(range(10)))) + list(ascii_lowercase)

    def __len__(self):
        """
        Calculates the length of the dataset-
        """
        return len(self.data)

    def __getitem__(self, idx):
        """
        Returns one sample (dict consisting of an image and its label)
        """

        # Read the image and labels
        image_path = os.path.join(self.root_dir, self.data[idx][1])
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        # Shape of the image should be H,W,C where C=1
        image = np.expand_dims(image, 0)
        # The label is the index of the class name in the list ['0','1',...,'9','a','b',...'z']
        # because we should have integer labels in the range 0-35 (for 36 classes)
        label = self.class_names.index(self.data[idx][0])

        sample = {'image': image, 'label': label}

        if self.transform:
            sample = self.transform(sample)

        return sample

# Implementations

## Data Loading

In [49]:
dataset = SignLangDataset(csv_file="labels.csv", root_dir=DATASET_PATH)

TRAINING_DATA_LENGTH = int(len(dataset) * 0.8)
TESTING_DATA_LENGTH = len(dataset) - TRAINING_DATA_LENGTH
training_data, testing_data = random_split(dataset, [TRAINING_DATA_LENGTH, TESTING_DATA_LENGTH])

def get_data_labels(dataset):
    data = list()
    labels = list()

    for item in dataset:
        data.append(item['image'])
        labels.append(item['label'])

    return np.array(data), np.array(labels)

X, Y = get_data_labels(training_data)
X = X.reshape(TRAINING_DATA_LENGTH, -1)

X_test, Y_test = get_data_labels(testing_data)
X_test = X_test.reshape(TESTING_DATA_LENGTH, -1)

## Decision Tree

In [32]:
from sklearn import tree

clf = tree.DecisionTreeClassifier(criterion='gini')

clf.fit(X, Y)

DecisionTreeClassifier()

### Score

In [33]:
print("Training Data:", clf.score(X, Y))
print("Testing Data:", clf.score(X_test, Y_test))

Training Data: 1.0
Testing Data: 0.39359504132231404


## Logistic Regression

In [36]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()
log_reg.fit(X, Y)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/thomas/Documents/Thomas/Uni/4_Semester/ML/sign_language/venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.1min finished


LogisticRegression(verbose=True)

### Score

In [37]:
print("Training Data:", log_reg.score(X, Y))
print("Testing Data:", log_reg.score(X_test, Y_test))

Training Data: 0.9939307851239669
Testing Data: 0.7143595041322314


## ANN

In [46]:
from sklearn.neural_network import MLPClassifier

In [50]:
clf = MLPClassifier(hidden_layer_sizes=(100,), verbose=True, random_state=1, max_iter=30)
X_norm = X / 255
clf.fit(X_norm, Y)

Iteration 1, loss = 2.75490898
Iteration 2, loss = 1.69653389
Iteration 3, loss = 1.29076086
Iteration 4, loss = 1.04849732
Iteration 5, loss = 0.89835993
Iteration 6, loss = 0.80091519
Iteration 7, loss = 0.71164187
Iteration 8, loss = 0.63526531
Iteration 9, loss = 0.56954650
Iteration 10, loss = 0.52567082
Iteration 11, loss = 0.48694625
Iteration 12, loss = 0.43616984
Iteration 13, loss = 0.40795641
Iteration 14, loss = 0.37649022
Iteration 15, loss = 0.35593887
Iteration 16, loss = 0.31807238
Iteration 17, loss = 0.29158045
Iteration 18, loss = 0.28239361
Iteration 19, loss = 0.25512628
Iteration 20, loss = 0.23507059
Iteration 21, loss = 0.21750258
Iteration 22, loss = 0.20767019
Iteration 23, loss = 0.19350298
Iteration 24, loss = 0.19390384
Iteration 25, loss = 0.16553587
Iteration 26, loss = 0.14776921
Iteration 27, loss = 0.14419917
Iteration 28, loss = 0.13015751
Iteration 29, loss = 0.12612691
Iteration 30, loss = 0.12022650


/Users/thomas/Documents/Thomas/Uni/4_Semester/ML/sign_language/venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (30) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(max_iter=30, random_state=1, verbose=True)

### Score

In [51]:
print("Training Data:", clf.score(X, Y))
print("Testing Data:", clf.score(X_test, Y_test))

Training Data: 0.9855371900826446
Testing Data: 0.8037190082644629
